<br>
<div align="center">
<font size="6"><b>ICS2205- Web Group Assignment Assigment</b></font><br><br>
<font size="5">Task 2: Text Analysis</font>
</div>

<font color="red"><b>Only run this notebook after trying to run "Task 2 Full dataset" and failing at the TF.IDF</b></font>

In [1]:
pip install flask flask==2.1.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --user -U nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
#NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords #List of stopwords
from nltk.stem import PorterStemmer #Stemming of words
#Other
import pandas as pd #Used to display TF.IDF
import numpy as np
import re
import random

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<font size ="5"><b>A. Process the news headline text</b></font> <br><br>

<font size ="4"><b>i. Parse JSON Files</b></font><br>
Each line in the provided dataset, is a JSON Object. By using the json package of python, each line in the data set is loaded and then added to a dictionary.

In [5]:
json_file_path = "NewsCategoryDataset_2017_2022.json"
limit = 10000
data = {}
i = 0

with open(json_file_path) as f:
    for line in f:
        if(line != '\n'):
            data[i] = json.loads(line)
            i += 1
        if(i==limit):
            break
    f.close()
# print(data[10]["headline"])
# print(data[10]["short_description"])
# print(data[10]["category"])

For analysis only the headline, category and short_description are required. Thus the following data is saved in its respective dictionaries. The "data'' dictionary is cleared to free up memory space.

In [6]:
headlineData = {} #Will store all headline data
categoryData = {} #Will store all category data

for i in range(len(data)):
    headlineData[i] = data[i]["headline"] + " " + data[i]["short_description"] #Stores the headline data
    categoryData[i] = data[i]["category"] #Stores the categories

data = {} #Free up Memory 

# print(headlineData[10])
# print(categoryData[10])

<font size ="4"><b>ii. Lexical Analyses</b></font><br>
Currently all the text found in headline data is in the form of a string. To be able to produce a TF.IDF matrix the text needs to be separated into strings containing a single word. This process is done using the “re” package [1]. Essentially for each of the headlines, the string is divided at each character that is not part of the alphabet.<br><br>
Case folding is also being performed at the same time to convert each of the characters in the string to lowercase. This is needed for the TF.IDF matrix to prevent having multiple distinct words representing the same word just with different casing.

In [7]:
for i in range(len(headlineData)): 
    token = [] # Will hold list of words
    tempString = headlineData[i]
    tokens = re.findall(r"[\w']+", tempString.casefold()) #Will tokenize the data by spaces and punct
    headlineData[i] = tokens
# print(headlineData[10])

<font size ="4"><b>iii. Stop Words & iv. Porter's Stemmer</b></font><br>
The headline data still currently contains a lot of commonly used words which are also referred to as stop words. These words are removed by comparing them with a list of stop words from the “NLTK” library [2]. If any words in the headline matches the stop words list the word is removed from the headline.<br><br>
Some words have similar meanings or are variations of a word which is their stem. Using the NLTK's library Porter's Stemmer [3] each word in the headline is reduced to its stem. 

In [8]:
ps = PorterStemmer() # Using NLTK library 
stop_words = set(stopwords.words('english')) #Using NLTK library

for i in range(len(headlineData)):
    filtered_sentence = []
    for w in headlineData[i]:
        if w not in stop_words: # Check word is not a stop word
            filtered_sentence.append(ps.stem(w)) # Reduce a word to its stem
    headlineData[i] = filtered_sentence
# print(headlineData[10])

In [9]:
headlineAmount = len(headlineData)
print("Amount of articles: ", headlineAmount)

Amount of articles:  10000


<font size ="5"><b> B. Construct a TF.IDF</b></font> <br><br>
<font size ="4"><b>Calculate Unique Words</b></font><br>
The distinct words in that form part of the headline data are found and put in a set. These words are going to be used for the TF.IDF and for extracting the data to JSON. There are a total of 36,120 distinct words in the headlines and short descriptions combined together. 

In [10]:
uniqueWords = set() #Holds every word present in the data
for i in range(len(headlineData)): #Loop through words
    for word in headlineData[i]:
        if word not in uniqueWords:
            uniqueWords.add(str(word))

print(len(uniqueWords))

18496


<font size ="4"><b>Term Frequency</b></font><br>
The term frequency is the frequency of a word in a single document with regards to the document’s length. This is obtained by counting each occurance of the word, then dividing the number of occurrences by the number of words in the document. [4]

In [11]:
def TermFrequency(doc,word):
    docLength = len(doc)
    
    occurance = 0
    for w in doc:
        if w == word:
            occurance += 1

    return occurance / docLength

<font size ="4"><b>Inverse Document Frequency</b></font><br>
The inverse document frequency(IDF) determines the frequency of a word throughout documents in a dataset. IDF is used to aid the score of words that are unique to specific documents. [5]

In [12]:
def CountDict(headlineData):
    
    countDict = {} #Will hold the number of times a word appears in distinct documents. Using the unique words a key and
                    #the number of times a word appears as the value for said key
        
    #Initiliaze all keys to 0 to avoid error and for simplicity
    for word in uniqueWords:
        countDict[word] = 0
        
    #Loop through all the words
    for i in range(headlineAmount):
        for word in headlineData[i]:
            usedWords = set() #usedWords will hold the words that have already be incremented for a specific document
                              #Its usage is to prevent increment the frequency for the same word in the same doc twice. 
            for word in headlineData[i]:
                if word not in usedWords:
                    usedWords.add(word)
                    countDict[word] += 1 #Increments frequency
    return countDict

wordFreq = CountDict(headlineData)
#print(dic)

In [13]:
def inverse_document_frequency(word):
    try:
        word_occurance = wordFreq[word] + 1
    except:
        word_occurance = 1
    return np.log(headlineAmount / word_occurance)

<font size ="4"><b>TF.IDF</b></font><br>
TF.IDF is used to search documents and extract the most important terms in relation to a specific document. The highest scoring words are considered the keywords for a specific document. [6]

In [14]:
def tf_idf(doc):
    vec = np.zeros((len(uniqueWords),))
    for word in doc:
        tf = TermFrequency(doc, word)
        idf = inverse_document_frequency(str(word))
        vec[uniqueWords_index[word]] = tf * idf
    return vec

In [15]:
uniqueWords_index = {}
for i, word in enumerate(uniqueWords):
    uniqueWords_index[word] = i

In [16]:
TF_IDF = []
for i in range(headlineAmount):
    TF_IDF.append(tf_idf(headlineData[i]))
# print(TF_IDF[10][uniqueWords_index["qatar"]])
# print(len(TF_IDF))
# print(len(uniqueWords_index))

<font size ="4"><b> Displaying the TF.IDF</b></font><br>
The TF.IDF is displayed using the panda dataframe. The tf idf is limited to the first 10 articles to be memory efficient.

In [17]:
tf_idf2 = pd.DataFrame(data = TF_IDF[:10])
tf_idf2.columns = uniqueWords_index
tfidf_matrix = tf_idf2.T
tfidf_matrix.columns = ['article'+ str(i) for i in range(1, 10+1)]
tfidf_matrix['count'] = tfidf_matrix.sum(axis=1)
tfidf_matrix = tfidf_matrix.sort_values(by ='count', ascending=False)
display(tfidf_matrix.drop(columns=['count']))

,article1,article2,article3,article4,article5,article6,article7,article8,article9,article10
23,0.000000,0.000000,0.368247,0.10880,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
sept,0.000000,0.000000,0.273951,0.16188,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
eaten,0.000000,0.000000,0.403081,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
pix11,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.327085,0.000000,0.000000,0.000000
isabel,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.308291,0.000000
...,...,...,...,...,...,...,...,...,...,...
say,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,-0.030854
year,0.000000,0.000000,0.000000,0.00000,0.0,-0.018032,0.000000,0.000000,0.000000,-0.020286
u,-0.013193,-0.014914,0.000000,0.00000,0.0,0.000000,0.000000,-0.017151,0.000000,-0.014293
said,-0.040892,0.000000,0.000000,0.00000,0.0,-0.039377,0.000000,0.000000,0.000000,0.000000


<font size ="5"><b>C. Extract Category Information</b></font> <br><br>
<font size ="4"><b>Calculate Unique Categories</b></font><br>
The unique categories are added to a set().

In [18]:
uniqueCategory = set() #Hold all the uniqueCategories 
for i in range(len(categoryData)):
    if categoryData[i] not in uniqueCategory:
        uniqueCategory.add(categoryData[i])
# print(len(uniqueCategory))

<font size ="4"><b>Group together headlines in the same category</b></font><br>
Using a dictionary, the headlines of a specific category are grouped. The index of the headline instead of the headline itself is stored to preserve memory. 

In [19]:
categoryHeadlines = {}  #Will hold all headlines index related to specific category
                        #Uses cateragory as a key, hold headlines indexes

for category in uniqueCategory: #Init Dictionary with empty list
    categoryHeadlines[category] = [] 

for i in range(headlineAmount): #Loop through all headlines and append them to relevant category
    categoryHeadlines[categoryData[i]].append(i)
    
#print(categoryHeadlines["SPORTS"][:50]) #Prints the first 50 headlines sports

<font size ="4"><b>i. Calculate the average weighting of a word</b></font><br>
Using the constructed TF.IDF, the average weighting of each word in a category is calculated. The weights are stored in a nested dictionary . The dictionary will have its parent key be the category and the sub-dictionary will have the words as keys and the value as scores.
<br><br>

<font size ="4"><b>ii. Get the highest-weighted n% of the terms for each category.</b></font><br>
For each category the only terms that are kept are the following: <br>
1. If the amount of terms is less than 100, all terms are kept. N = Amount of terms
1. If the amount of terms is between 100 and 1000, the top 100 terms are kept. N = 100
1. If the amount of terms is greater than 1000, the top 10% of terms are kept. N% = 10%

In [20]:
categoryDictionary = {} #Will hold nested Dictionaries

for category in categoryHeadlines:#For each category
    tempDict = {} #Tempory dictionary to hold weighting of words
    
    for word in uniqueWords:
        tempDict[word] = 0 #Initialize keys
        
    for index in categoryHeadlines[category]: #For each headline in each category
        for word in headlineData[index]:
            tempDict[word] += TF_IDF[index][uniqueWords_index[word]] #Add the word importance for that document
    
    for word in uniqueWords:
        if tempDict[word] == 0: 
            tempDict.pop(word) #Remove any word that where not used in a specific category
        else:
            tempDict[word] = tempDict[word]/len(categoryHeadlines[category]) #Normalise to avoid large scores in larger categories
            tempDict[word] *= 100 #Uniformal Increae to make score differences more noticable
    
    #ii. Get the highest-weighted n% of the terms for each category.
    
    #Reduce to N% of values
    sortedDict = sorted(tempDict.items(), key=lambda x:x[1], reverse = True) # Sort category
    termAmount = len(sortedDict)
    
    #If Terms <= 100 do nothing

    #If Terms > 100 & less < 1000  reduce to top 100
    if(termAmount > 100) and (termAmount < 1000):
        sortedDict = sortedDict [:100] #Reducing to the top 100
    w
    #If Terms > 1000 reduce to top 10%
    if(termAmount > 1000):
        nTerms = round(0.1 * termAmount)
        sortedDict = sortedDict [:nTerms]
            
    categoryDictionary[category] = dict(sortedDict)
# print(categoryDictionary["SPORTS"])

In [21]:
#Convert to correct format for keyword cloud

categoryDictionary2 = {}
for key in categoryDictionary:
    categoryDictionary2[key] = [] #List of dictionaries
    for nestedKey in categoryDictionary[key]:
        categoryDictionary2[key].append({"text": nestedKey, "size": categoryDictionary[key][nestedKey]}) #Keyword Format
    
categoryDictionary = categoryDictionary2

<font size ="4"><b>iii. Extracting all the data into JSON files</b></font><br>

Re-open the data files containing all the data and store data in a dictionary.

In [22]:
data = {}
i = 0
with open(json_file_path) as f:
    f.seek(0)
    for line in f:
        if(line != '\n'):
            data[i] = json.loads(line)
            i += 1
f.close()

In [23]:
categorizedData = {} # Getting the headlines data instead of indexes in the dictionary
for category in uniqueCategory:
    headlinesIndexes = categoryHeadlines[category]
    categorizedData[category] = {}
    for index in headlinesIndexes:
        categorizedData[category][index] = data[index] 

categoryJsonData = { } will hold all the article data grouped by categories and has the following structure:
* Category
    * listOfArticles
        * (Article Data)
    * top_terms
        * text
        * size

categoryBubbleJsonData = [ ] will hold the data neccassary to construct the bubble chart for the bubble graph and contains a list of dictionaries of the following structure:
* name (The name of category)
* score (Number of articles in the category)

In [24]:
categoryJsonData = {}
categoryBubbleJsonData = []
#categoryJsonData[category][list]/top term
for category in uniqueCategory:
    categoryJsonData[category] = {}

for category in uniqueCategory:
    categoryJsonData[category]["listOfArticles"] = categorizedData[category]
    categoryJsonData[category]["top_terms"] = categoryDictionary[category]


for category in uniqueCategory:
    tempDic = {}
    tempDic["name"] = category 
    tempDic["score"] = len(categoryJsonData[category]["listOfArticles"])
    categoryBubbleJsonData.append(tempDic)
    
# print(categoryBubbleJsonData)

Using the json library, the dictionaries are writen to their respective files
* categoryData.json
* categoryBubbleData.json

In [25]:
with open("categoryData.json","w") as outfile:
    json.dump(categoryJsonData,outfile)
outfile.close()

with open("static/categoryBubbleData.json","w") as outfile:
       json.dump(categoryBubbleJsonData,outfile)
outfile.close() 

<font size ="5"><b>D. K-means</b></font> <br><br>
Generate a set of random cluster nodes to start clustering. The number of nodes (k) = number of categories.

In [26]:
clusterNodes = [] #Contains Headline Indexes
for i in range(len(uniqueCategory)):
    randomNum = random.randint(0,headlineAmount)
    if(randomNum not in clusterNodes):
        clusterNodes.append(randomNum)
    else:
        i -= 1
# print(clusterNodes)

<b>Cosine Similarity Function</b>: Used to compare the similarity between two documents.

In [27]:
def cosineSimilarity(node, Y_set):
    c = 0
    l1 =[]
    l2 =[]
    X_set = {}
    rvector = []

    X_set = {w for w in headlineData[node]} #Grab Node
    rvector = X_set.union(Y_set) #Union Them

    for w in rvector:
        if w in X_set: 
            l1.append(TF_IDF[node][uniqueWords_index[w]]) #Value of Wieghting for that document
        else: 
            l1.append(0)

        if w in Y_set:
            l2.append(1)
        else: 
            l2.append(0)

    # cosine formula 
    for j in range(len(rvector)):
        c+= l1[j]*l2[j]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

<b>kMeans Function</b>: Clusters the headline data into clusters. It will iterate through every headline for each node. While iterating it will check the cosine similarity of the headline with the current node and store for each headline, the node it's the most similiar to.

In [ ]:
def kMeansClustering(startingNodes,maxIt):
    change = True # Flags if a change occured in the nodes between one iteration and the next
    iterations = 0 # The number of current iteration 
    nodeAmount = len(clusterNodes) # The number of clusters
    
    while (change == True) and (iterations < maxIt): # If there was a change and max iterations not reach
        kMeansDic = {} 
        totalCosineDic = {}
        meanCosineDic = {}
        index = 0
        testing = 0
        
        for nd in clusterNodes: #For every k, create and empty list
            kMeansDic[nd] = []
            totalCosineDic[nd] = 0 #Initialize co-sine similarity to 0
            
        for headline in headlineData:
            Y_set = {w for w in headlineData[headline]} #Query == Entire headline
            sim = [] 
            
            bestCosine = 0 #Best Cosine Similirity
            bestNode = -10 #Keeps track which node the document is closest too
            totalCosine = 0 #Reset the total Cosine
            
            for node in clusterNodes:
                cosine = cosineSimilarity(node, Y_set) # Get Cosine Similiarity
            
                if(cosine > bestCosine): #Store best Cosine
                    bestNode = node
                    bestCosine = cosine
        
            #Catches Nodes that don't get assigned a cluster
            if (bestNode == -10):
                randomNum = random.randint(0,nodeAmount-1)
                kMeansDic[clusterNodes[randomNum]].append((index,bestCosine))
                testing += 1
            else:
                kMeansDic[bestNode].append((index,bestCosine))
                totalCosineDic[bestNode] += bestCosine
            index += 1
        
        iterations += 1
        if(iterations != maxIt): #Unless Limit not reach
            change = False
            for j in range(len(clusterNodes)): #For each starting node
                node = clusterNodes[j] 
                total = totalCosineDic[node]
                length = len(kMeansDic[node])
                mean = total/length # Calculate mean of new node
                
                bestFit = 100
                bestIndex = node
                
                for tup in kMeansDic[node]: #Search to find new node with closest similarity
                    diff = abs(tup[1]-mean) #Difference with the cosine similarity of a headline and the mean
                    
                    if(diff < bestFit): 
                        bestFit = tup[1]
                        bestIndex = tup[0]
                        
                if(node != bestIndex): #If a change occurred
                    clusterNodes[j] = bestIndex
                    change = True
        else:
            return kMeansDic
maxIt = 10
kMeansDic = kMeansClustering(clusterNodes,maxIt)

<font size ="5"><b>E. Extract K-Means Information</b></font><br><br>
<font size ="4"><b>i. Calculate the average weighting of a word</b></font><br>
Using the constructed TF.IDF, the average weighting of each word in a cluster is calculated. The weights are stored in the a nested dictionary. The dictionary will have its parent key be the cluster node and the sub-dictionary will be have the words as keys and the value as scores.<br><br>

<font size ="4"><b>ii. Get the highest-weighted n% of the terms for each category.</b></font><br>
For each cluster the only terms that are kept are the following: <br>
1. If the amount of terms is less than 100, all terms are kept. N = Amount of terms
1. If the amount of terms is between 100 and 1000, the top 100 terms are kept. N = 100
1. If the amount of terms is greater than 1000, the top 10% of terms are kept. N% = 10%

In [ ]:
KMeansDictionary = {} #Will hold nested Dictionaries

for node in kMeansDic:#For each category
    tempDict = {} #Tempory dictionary to hold weighting of words
    
    for word in uniqueWords:
        tempDict[word] = 0 #Initialize keys
        
    for tup in kMeansDic[node]: #For each headline in each cluster
        index = tup[0]
        for word in headlineData[index]:
            tempDict[word] += TF_IDF[index][uniqueWords_index[word]] #Add the word importance for that document
    
    for word in uniqueWords:
        if tempDict[word] == 0: 
            tempDict.pop(word) #Remove any word that where not used in a specific cluster
        else:
            tempDict[word] = tempDict[word]/len(kMeansDic[node]) #Normalise to avoid large scores in larger categories
            tempDict[word] *= 100 #Uniformal Increae to make score differences more noticable
    
    #ii. Get the highest-weighted n% of the terms for each category.
    
    #Reduce to N% of values
    sortedDict = sorted(tempDict.items(), key=lambda x:x[1], reverse = True) # Sort category
    termAmount = len(sortedDict)
    
    #If Terms <= 100 do nothing

    #If Terms > 100 & less < 1000  reduce to top 100
    if(termAmount > 100) and (termAmount < 1000):
        sortedDict = sortedDict [:100] #Reducing to the top 100
    w
    #If Terms > 1000 reduce to top 10%
    if(termAmount > 1000):
        nTerms = round(0.1 * termAmount)
        sortedDict = sortedDict [:nTerms]
            
    KMeansDictionary[node] = dict(sortedDict)

In [ ]:
print(KMeansDictionary)

In [ ]:
#Convert to correct format for keyword cloud

KMeansDictionary2 = {}
for key in KMeansDictionary:
    KMeansDictionary2[key] = [] #List of dictionaries
    for nestedKey in KMeansDictionary[key]:
        KMeansDictionary2[key].append({"text": nestedKey, "size": KMeansDictionary[key][nestedKey]}) #Keyword Format

KMeansDictionary = KMeansDictionary2
clusterDictionary2 = None

<font size ="4"><b>iii. Extracting all the data into JSON files</b></font><br>

Re-open the data files containing all the data and store data in a dictionary

In [ ]:
data = {}
i = 0
with open(json_file_path) as f:
    f.seek(0)
    for line in f:
        if(line != '\n'):
            data[i] = json.loads(line)
            i += 1
f.close()

In [ ]:
clusterData = {} # Getting the headlines data instead of indexes in the dictionary
for node in KMeansDictionary:

    headlinesIndexes = kMeansDic[node]
    clusterData[node] = {}
    for tup in headlinesIndexes:
        index = tup[0]
        clusterData[node][index] = data[index] 

clusterJsonData = { } will hold all the headline data grouped by cluster nodes and has the following structure:
* ClusterNode
    * listOfArticles
        * (Article Data)
    * top_terms
        * text
        * size

clusterBubbleJsonData = [ ] will hold the data neccassary to construct the bubble chart for the bubble graph and contains a list of dictionaries of the following structure:
* name (The index of the document currently being a centeriod)
* score (Number of articles in the cluster)

In [ ]:
clusterJsonData = {}
clusterBubbleJsonData = []
#categoryJsonData[category][list]/top term
for node in KMeansDictionary:
    clusterJsonData[node] = {}

for node in KMeansDictionary:
    clusterJsonData[node]["listOfArticles"] = clusterData[node]
    clusterJsonData[node]["top_terms"] = KMeansDictionary[node]

for node in KMeansDictionary:
    tempDic = {}
    tempDic["name"] = str(node)
    tempDic["score"] = len(clusterJsonData[node]["listOfArticles"])
    clusterBubbleJsonData.append(tempDic)

In [ ]:
with open("clusterData.json","w") as outfile:
    json.dump(clusterJsonData,outfile)
outfile.close()

with open("static/clusterBubbleData.json","w") as outfile:
       json.dump(clusterBubbleJsonData,outfile)
outfile.close() 

<div align="center">
<font size="5">Task 2: Web Application</font>
</div>

<font size ="5"><b>A. Set up Flask</b></font> <br><br>
<font size ="4"><b>ii. Import Json Files</b></font> 

In [ ]:
json_file_path = "NewsCategoryDataset_2017_2022.json" #Raw JSON File

data = {}
i = 0

with open(json_file_path) as f:
    for line in f:
        if(line != '\n'):
            data[i] = json.loads(line)
            i += 1
    f.close()

In [ ]:
json_file_path = "categoryData.json" #Category Data JSON File

categData = {}
i = 0

with open(json_file_path) as f:
    categData = json.load(f)
    f.close()
# print(categData["SPORTS"])

In [ ]:
json_file_path = "clusterData.json" #Cluster Data JSON File

clustData = {}
i = 0

with open(json_file_path) as f:
    clustData = json.load(f)
    f.close()

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html',data = data)

@app.route("/<int:index>")
def article(index):
    index = int(index)
    return render_template('article.html', art_id = index, headline = data[index]["headline"],
                           short_description = data[index]["short_description"],date = data[index]["date"],
                           link = data[index]["link"])
@app.route('/category')
def category():
    return render_template('category.html')

@app.route('/cluster')
def cluster():
    return render_template('cluster.html')

@app.route('/cluster/<clust>')
def clust(clust):
    keywordData = clustData[clust]["top_terms"]
    return render_template('clust.html', data = clustData[clust], keywordData = keywordData)

@app.route('/category/<categ>')
def categ(categ):
    categ = categ.upper()
    keywordData = categData[categ]["top_terms"]
    return render_template('categ.html', data = categData[categ], keywordData = keywordData)

In [ ]:
if __name__ == "__main__":
    app.run()

<font size ="4"><b><font size ="4"><b>iii. Show the list of documents as a ‘clickable’ list</b></font> </b></font><br>
This is the home page (index.html) of Flask server.

<img src= "images/All Articles.png">

<div align=center><b>Figure 1:</b> List of clickable headlines</div>

<font size ="5"><b>B. Show headline information</b></font> <br>

<b>Instructions:</b>
Click on any of the links in figure 1 to open up the headline. The url will indicate the headline you are currently visiting. You can visit a specific url by adding the index number to default url such as: http://127.0.0.1:5000/10

<img src= "images/Headline_Info.png">

<div align=center><b>Figure 2:</b> An example of a headline</div>

<font size ="5"><b>C. Show the list of categories as an interactive bubble chart.</b></font> <br><br>

Bubble Chart were constructed using D3.js[7]<br>
Keyword Clouds were constructed using Easy Word Cloud using D3[8]<br>

<b>Instructions:</b>
This page can be reached by adding category to the default url such as this: http://127.0.0.1:5000/category

A specific category can be accessed by adding category followed by the name of the category to defualt url like the following: http://127.0.0.1:5000/category/POLITICS

<img src= "images/categoryBubbleChart.png">

<div align=center><b>Figure 3:</b> Category Bubblechart</div>

<img src= "images/PoliticsExample.png">

<div align=center><b>Figure 4:</b> Expaning the bubble "Politics"</div>

<font size ="5"><b>D. Show the list of clusters as an interactive bubble chart.</b></font> <br><br>
<b>Instructions:</b>
This page can be reached by adding category to the default url such as this: http://127.0.0.1:5000/cluster

A specific category can be accessed by adding category followed by the name of the category to defualt url: http://127.0.0.1:5000/cluster/23856

<img src= "images/clusterBubbleChart.png">

<div align=center><b>Figure 5:</b> Cluster Bubblechart</div>

<img src= "images/23856Example.png">

<div align=center><b>Figure 6:</b> Expanding the bubble: 23856</div>

<font size ="5"><b>Reference List</b></font>

[1] re — Regular expression operations, Python. https://docs.python.org/3/library/re.html (accessed 28/01/2023) <br>
[2] NLTK stop words, Python. https://pythonspot.com/nltk-stop-words/ (accessed 28/01/2023) <br>
[3] Sample usage for stem, NLTK. https://www.nltk.org/howto/stem.html (accessed 28/01/2023) <br>
[4] K.Ganesan , What is Term Frequency? Opinosis Analytics. https://www.opinosis-analytics.com/knowledge-base/term-frequency-explained/#:~:text=TF(t)%20%3D%20(Number,of%20terms%20in%20the%20document). (accessed 28/01/2023) <br>
[5] K.Ganesan, What is Inverse Document Frequency (IDF)? Kavita Ganesan. https://kavita-ganesan.com/what-is-inverse-document-frequency/#.Y9KclnbMKUk  (accessed 28/01/2023) <br>
[6] B.Stecanella, Understanding TF-IDF: A Simple Introduction, Monkey Learn. https://monkeylearn.com/blog/what-is-tf-idf/   (accessed 28/01/2023) <br>
[7] F.Almasi, How to Make Interactive Bubble Charts in D3.js, Webtips.  https://www.webtips.dev/how-to-make-interactive-bubble-charts-in-d3-js (accessed by 28/01/2023) <br>
[8] wvengen, Easy Word Cloud using D3, Github. https://github.com/wvengen/d3-wordcloud (accessed 28/01/2023) <br>